In [1]:
#바뀜 확인
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
#import missingno as msno
#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
path = './001.data/'

In [3]:
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
specs_df = pd.read_csv(path + 'specs.csv')
train_labels_df = pd.read_csv(path +'train_labels.csv')
sub_df = pd.read_csv(path +'sample_submission.csv')

FileNotFoundError: [Errno 2] File b'./001.data/train.csv' does not exist: b'./001.data/train.csv'

In [ ]:
train_df.shape, test_df.shape, specs_df.shape,train_labels_df.shape

In [ ]:
pd.options.display.max_columns = 100

### user func

In [ ]:
def info_df(data):
    '''
    data의 type, null_count, null_rate를 알려주는 함수 
    추가로 unique한 데이터의 수를 알려줌 (속도가 느리므로 필요시 주석처리 필요)
    '''
    info_df = pd.DataFrame({"type":data.dtypes,
                            'null_count':data.isnull().sum(),
                           'null_rate':data.isnull().sum()/data.isnull().count() * 100})
    
#     cols = data.columns.values
    
#     uni_count =[]
#     for col in cols:
#         uni_count.append(len(data[col].unique()))
#     info_df['uni_count'] = uni_count
    
    return info_df

In [ ]:
#feature importance
def model_fi(model):
    fi_df = pd.DataFrame({'importance': model.feature_importances_},index = train_X.columns)
    fi_df = fi_df.sort_values(by ='importance', ascending=False)
    fi_df['importance'] = round(fi_df['importance'],4)
    
    return fi_df

### merge func

In [ ]:
def GS_event_G(data):
    '''
    train_df/test_df를 넣으면 game_session별로 묶어진 event_code별 카운트를 반환
    NaN은 0으로 체움
    '''
    data['values'] = 1
    GS_event = data.pivot_table(index = 'game_session', columns = 'event_code', values ='values',aggfunc= 'sum')
    GS_event.fillna(0,inplace= True)
    GS_event = GS_event.reset_index()
    
    return GS_event

In [ ]:
%%time
GS_event = GS_event_G(train_df)
t_GS_event = GS_event_G(test_df)

In [ ]:
GS_event.shape, t_GS_event.shape

In [ ]:
%%time
def GS_max_G(data,data2):
    '''
    data에 train_df/test_df를 넣으면 game_session별로 묶어진 timestamp,event_count,game_time,type의 max를 반환
    data2에 GS_event/t_GS_event를 넣으면 merge 
    다소오래걸림
    '''
    #max값 산출
    GS_max= data.groupby('game_session')["timestamp","installation_id","event_count","game_time","type"].max()
    GS_max = GS_max.reset_index()
    GS_max = GS_max.rename(columns={"timestamp":"timestamp_max"})
    
    #min값 산출
    GS_min = data.groupby('game_session')["timestamp"].min()
    GS_min = GS_min.reset_index()
    GS_min = GS_min.rename(columns={"timestamp":"timestamp_min"})
    
    #min,max 결합
    GS_m= pd.merge(GS_max,GS_min)
    GS_m = GS_m.sort_values(['installation_id','timestamp_max'])
    
    #GS_m과,GS_event의 결합
    GS_me = pd.merge(GS_m,data2)
    
    return GS_me


In [ ]:
%%time
GS_me = GS_max_G(train_df,GS_event)
t_GS_me = GS_max_G(test_df,t_GS_event)

In [ ]:
GS_me.shape, t_GS_me.shape

In [ ]:
def GS_title_G(data,col1,col2):
    '''
    data를 넣으면 game_session이 속한 title/world을 1로 표시하여 merge
    result는 world를 받고
    result2는 title을 받는걸 권장
    '''
    data['values'] = 1
    result = pd.DataFrame()
    result2 = pd.DataFrame()
    
    #world
    result = data.pivot_table(index = 'game_session',
                                      columns= col1,
                                      values = 'values',
                                      aggfunc= 'count')
    for col in result.columns.values:
        result[col] = result[col].apply(lambda x : 1 if x>0 else 0)
    
    #title
    result2 = data.pivot_table(index = 'game_session',
                                  columns= col2,
                                  values = 'values',
                                  aggfunc= 'count')
    for col in result2.columns.values:
        result2[col] = result2[col].apply(lambda x : 1 if x>0 else 0)
    
    final = pd.merge(result,result2, right_index= True, left_index = True)
    final = final.reset_index()
    return final

In [ ]:
%%time
GS_title = GS_title_G(train_df, "world","title")
t_GS_title = GS_title_G(test_df, "world","title")

In [ ]:
GS_title.shape, t_GS_title.shape

In [ ]:
GS_met= pd.merge(GS_me,GS_title)
t_GS_met= pd.merge(t_GS_me,t_GS_title)

In [ ]:
GS_met.shape, t_GS_met.shape

In [ ]:
%%time
GS_labels = pd.merge(GS_met,train_labels_df[["game_session","accuracy_group"]],how='left',on= 'game_session')
GS_labels = GS_labels.sort_values(['installation_id','timestamp_max'])

In [ ]:
GS_labels.shape

In [ ]:
def to_datetime(data,col1,col2):
    data[col1] = pd.to_datetime(data[col1])
    data[col2] = pd.to_datetime(data[col2])
    return data

In [ ]:
%%time
GS_time = to_datetime(GS_labels,"timestamp_max","timestamp_min")
t_GS_time = to_datetime(t_GS_met,"timestamp_max","timestamp_min")

In [ ]:
GS_time.shape,t_GS_time.shape

In [ ]:
def make_wide(data,data2, ):
    '''
    GS_labels와 GS_event를 merge한 data을 넣으면 wide 포맷으로 변환
    1차 for문은 unique한 installment_id를 반복하기 위해
        2차 for문은 각 assessemnt의 위치를 반환하기 위함
    최종적으로 생략했던 train_label_df와 병합
       
    '''
    #기본변수선언
    # 고유 installation_id 
    uni_iids = data.installation_id.unique()

    #game_session(assessment)별 결과를 임시로 담아놓을 장소
    world_ls = list(t_GS_title.iloc[:,1:].columns.values)
    event_ls = list(t_GS_event.iloc[:,1:].columns.values)
    base_ls = ['installation_id','game_session','event_count',
                  'Activity_c','Assessment_c' ,'Clip_c', 'Game_c',
                  'Activity_t','Assessment_t' ,'Clip_t', 'Game_t',
                  'timestamp_max','timestamp_min','time_gap','game_time','ass_gt']
    result_col = base_ls + event_ls + world_ls
    
    pure_result = pd.DataFrame(columns= result_col, 
                               data= [[0] * len(result_col)])
    pure_result['time_gap'] = datetime.timedelta(0)
    
    #type별 count_time을 보관하기 위한 장소 
    type_df = pd.DataFrame(columns = ['Activity','Assessment' ,'Clip', 'Game'],
                           data = [[0,0,0,0]])
    #event_code별, game_time 등 sum값을 보관하기 위한 장소
    sum_list = data.columns.drop(['installation_id','type','game_session',
                                  'timestamp_max','timestamp_min','accuracy_group']).values
    
    #최종결과가 담길 장소
    result_df =pd.DataFrame()
    
    #Assessment의 범위 설정
    n = 0
    m = 0

    for uni_iid in uni_iids:
        as_indexs = data.loc[(data.installation_id == uni_iid)&
                              (data.type == 'Assessment')&
                              (data.accuracy_group.notnull())].index.values
        last_index = data[data.installation_id == uni_iid].index.max()

        for as_index in as_indexs:
            result = pure_result
            m = as_index
            try: 
                #type count
                type_dict = data.iloc[n:m].groupby('type')['installation_id'].count().to_dict()
                temp = (type_df + pd.DataFrame([type_dict])).fillna(0)
                temp.columns = ['Activity_c','Assessment_c' ,'Clip_c', 'Game_c']
                result[['Activity_c','Assessment_c' ,'Clip_c', 'Game_c']] = temp

                #type time
                type_dict = data.iloc[n:m].groupby('type')['game_time'].sum().to_dict()
                temp = (type_df + pd.DataFrame([type_dict])).fillna(0)
                temp.columns = ['Activity_t','Assessment_t' ,'Clip_t', 'Game_t']
                result[['Activity_t','Assessment_t' ,'Clip_t', 'Game_t']] = temp

                #time_gap   
                result['timestamp_max'] = data.iloc[m].timestamp_max
                result['timestamp_min'] = data.iloc[n].timestamp_min
                time_gap = data.iloc[m].timestamp_max - data.iloc[n].timestamp_min
                result['time_gap'] =time_gap 

#                 #ass_gametime 
#                 result['ass_gt'] = data.loc[m].game_time

                #event_code별, game_time 등의 sum 
                sum_df = data[n:m].groupby('installation_id')[sum_list].sum().reset_index().fillna(0)
                sum_df = sum_df.drop('installation_id',axis= 1)
                result[sum_df.columns.values] = sum_df 

                #id
                result['installation_id'] = uni_iid
                result['game_session'] = data.iloc[m].game_session

                #result
                result_df= result_df.append(result)
            except Exception as e:
                #처음이 assessment인 경우 에러발생으로 인해 기본갑 0 부여 
                print(e)
                result['installation_id'] = uni_iid
                result['game_session'] = data.iloc[m].game_session

                result_df= result_df.append(result)
                
        #각 installment_id의 가장 마지막 row에 +1
        n= last_index+1
    result_df.fillna(0,inplace=True)   
        
    #최종적으로 생략되었던 train_labels 부분 추가(accuracy_group 등)
    wide = pd.merge(result_df,data2.drop("installation_id",axis= 1))
    
    return wide

In [ ]:
def test_make_wide(data):
    '''
    train_df와 달리 마지막 assessement까지의 데이터를 결합
       
    '''
    #기본변수선언
    # 고유 installation_id 
    uni_iids = data.installation_id.unique()

    #game_session(assessment)별 결과를 임시로 담아놓을 장소
    world_ls = list(t_GS_title.iloc[:,1:].columns.values)
    event_ls = list(t_GS_event.iloc[:,1:].columns.values)
    base_ls = ['installation_id','game_session','event_count',
                  'Activity_c','Assessment_c' ,'Clip_c', 'Game_c',
                  'Activity_t','Assessment_t' ,'Clip_t', 'Game_t',
                  'timestamp_max','timestamp_min','time_gap','game_time','ass_gt']
    result_col = base_ls + event_ls + world_ls

    
    pure_result = pd.DataFrame(columns= result_col,
                               data= [[0] * len(result_col)])
    pure_result['time_gap'] =datetime.timedelta(0)
    
    #type별 count_time을 보관하기 위한 장소 
    type_df = pd.DataFrame(columns = ['Activity','Assessment' ,'Clip', 'Game'],
                           data = [[0,0,0,0]])
    #event_code별, game_time 등 sum값을 보관하기 위한 장소
    sum_list = data.columns.drop(['installation_id','type','game_session',
                                  'timestamp_max','timestamp_min']).values
    
    #최종결과가 담길 장소
    result_df =pd.DataFrame()
    
    #Assessment의 범위 설정
    n = 0
    m = 0

    for uni_iid in uni_iids:
        as_indexs = data.loc[(data.installation_id == uni_iid)&
                              (data.type == 'Assessment')].index.values
        last_index = data[data.installation_id == uni_iid].index.max()

        result = pure_result
        m = as_indexs[-1]
        try: 
            #type count
            type_dict = data.iloc[n:m].groupby('type')['installation_id'].count().to_dict()
            temp = (type_df + pd.DataFrame([type_dict])).fillna(0)
            temp.columns = ['Activity_c','Assessment_c' ,'Clip_c', 'Game_c']
            result[['Activity_c','Assessment_c' ,'Clip_c', 'Game_c']] = temp

            #type time
            type_dict = data.iloc[n:m].groupby('type')['game_time'].sum().to_dict()
            temp = (type_df + pd.DataFrame([type_dict])).fillna(0)
            temp.columns = ['Activity_t','Assessment_t' ,'Clip_t', 'Game_t']
            result[['Activity_t','Assessment_t' ,'Clip_t', 'Game_t']] = temp

            #time_gap   
            result['timestamp_max'] = data.iloc[m].timestamp_max
            result['timestamp_min'] = data.iloc[n].timestamp_min
            time_gap = data.iloc[m].timestamp_max - data.iloc[n].timestamp_min
            result['time_gap'] =time_gap 

            #ass_gametime 
            result['ass_gt'] = data.loc[m].game_time

            #event_code별, game_time 등의 sum 
            sum_df = data[n:m].groupby('installation_id')[sum_list].sum().reset_index()
            sum_df = sum_df.drop('installation_id',axis= 1)
            result[sum_df.columns.values] = sum_df 

            #id
            result['installation_id'] = uni_iid
            result['game_session'] = data.iloc[m].game_session

            #result
            result_df= result_df.append(result)
        except Exception as e:
            #처음이 assessment인 경우 에러발생으로 인해 기본갑 0 부여 
            print(e)
            result['installation_id'] = uni_iid
            result['game_session'] = data.iloc[m].game_session

            result_df= result_df.append(result)
                
        #각 installment_id의 가장 마지막 row에 +1
        n= last_index+1
    result_df.fillna(0,inplace=True)   
    
    return result_df


In [ ]:
%%time
wide = make_wide(GS_time,train_labels_df)
t_wide = test_make_wide(t_GS_time)

In [ ]:
wide.to_csv('wide.csv',index=False)
t_wide.to_csv('t_wide.csv',index=False)

In [ ]:
wide.shape, t_wide.shape

In [ ]:
wide.head()

In [ ]:
t_wide.head()

In [ ]:
#test set title추가
t_wide = pd.merge(t_wide,test_df[['game_session','title']])

### change type

In [ ]:
def one_hot_col(data,col):
    '''
    one_hot인코딩해줌
    '''
    dump = pd.get_dummies(data[col])
    dump.columns=['Bird Measurer (Assessment)_f','Cart Balancer (Assessment)_f',
                  'Cauldron Filler (Assessment)_f','Chest Sorter (Assessment)_f',
                  'Mushroom Sorter (Assessment)_f']
    data=pd.concat([data, dump], axis=1)
    return data

In [ ]:
wide_ohe = one_hot_col(wide,'title')
t_wide_ohe = one_hot_col(t_wide,'title')

In [ ]:
def date_to_s(data,col):
    data[col+"_s"] = data[col].apply(lambda x: x.seconds)
    return data

In [ ]:
wide_s = date_to_s(wide_ohe,'time_gap')
t_wide_s = date_to_s(t_wide_ohe,'time_gap')

### final set

In [ ]:
list_drop = ['installation_id','game_session','event_count','Clip_t','time_gap',
             'timestamp_max', 'timestamp_min','title','accuracy','num_correct', 'num_incorrect','ass_gt']
list_drop2 = ['installation_id','game_session','event_count','Clip_t','time_gap',
             'timestamp_max', 'timestamp_min','title','ass_gt']

In [ ]:
wide_fin=wide_s.drop(list_drop,axis=1)
t_wide_fin=t_wide_s.drop(list_drop2,axis=1)

In [ ]:
wide_fin.columns.values

In [ ]:
t_wide_fin.columns.values

In [ ]:
wide_fin.shape,t_wide_fin.shape

### 상관관계분석

In [ ]:
heat_sameple = wide_fin
heat_sameple = pd.concat([wide_fin.accuracy_group,heat_sameple],axis=1)
plt.figure(figsize= (10,10))
sns.heatmap(heat_sameple.corr(),vmax = 1, vmin= -1,cmap="RdYlBu")
plt.show()

In [ ]:
heat_sameple = wide_fin.iloc[:,20:40]
heat_sameple = pd.concat([wide_fin.accuracy_group,heat_sameple],axis=1)
plt.figure(figsize= (10,10))
sns.heatmap(heat_sameple.corr(),vmax = 1, vmin= -1,cmap="RdYlBu",annot= True)
plt.show()

### 2차 모델 model

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

In [ ]:
y = wide_fin.accuracy_group
X = wide_fin.drop('accuracy_group',axis= 1)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.2, random_state = 42)

In [ ]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

In [ ]:
model_1 = lgbm.LGBMClassifier()

In [ ]:
model_1.fit(train_X,train_y)

In [ ]:
model_1.score(train_X,train_y)

In [ ]:
model_1.score(test_X,test_y)

In [ ]:
pred_y = model_1.predict(test_X)

In [ ]:
from sklearn.metrics import classification_report
from  sklearn.metrics import precision_recall_fscore_support

In [ ]:
def report_df(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-1] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [ ]:
report_df(test_y,pred_y)

In [ ]:
model_fi(model_1)

### real mode

In [ ]:
y = wide_fin.accuracy_group
X_train = wide_fin.drop('accuracy_group',axis= 1)
X_test = t_wide_fin

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.columns

In [ ]:
X_test.columns

In [ ]:
model_1 = lgbm.LGBMClassifier()

In [ ]:
model_1.fit(X_train,y)

In [ ]:
model_1.score(X_train,y)

In [ ]:
model_fi(model_1)

In [ ]:
pred_y = model_1.predict(X_test)

### 1차제출파일

In [ ]:
pred_df = pd.DataFrame()

In [ ]:
pred_df['installation_id'] = t_wide_s.installation_id
pred_df['accuracy_group'] = pred_y

In [ ]:
sub_d = sub_df.drop('accuracy_group',axis =1)
submission = pd.merge(sub_d,pred_df)

In [ ]:
submission.to_csv('submission.csv',index= False)